# Conversion of the dataset into predicates

In [1]:
import json
import os
import random
from IPython.display import display
from pyswip import *
import subprocess
import pandas as pd

## Loading training and validation scenes and questions

In [2]:
with open("/media/murali/My Passport/GQA/sceneGraphs/train_sceneGraphs.json", 'r') as file:
    train_scene = json.load(file)

with open("/media/murali/My Passport/GQA/sceneGraphs/val_sceneGraphs.json", 'r') as file:
    val_scene= json.load(file)
    
with open("/media/murali/My Passport/GQA/questions1.2/train_balanced_questions.json", 'r') as file:
    train_ques = json.load(file)

with open("/media/murali/My Passport/GQA/questions1.2/val_balanced_questions.json", "r") as file:
    val_ques = json.load(file)

## Enumerating the Semantic operations

In [3]:
all_sem_ops = []

for qkey in train_ques:
    question = train_ques[qkey]
    sem_ops = question["semantic"]
    for sem_op in sem_ops:
        all_sem_ops.append(sem_op["operation"])

for qkey in val_ques:
    question = val_ques[qkey]
    sem_ops = question["semantic"]
    for sem_op in sem_ops:
        all_sem_ops.append(sem_op["operation"])
        
all_sem_ops = sorted(list(set(all_sem_ops)))

print("Total semantic operations: {}".format(len(all_sem_ops)))

Total semantic operations: 136


## Enumerating the Objects, Relations and Attributes

In [4]:
all_relations = []
all_attributes = []
all_objects = []

for skey in train_scene:
    objects = train_scene[skey]["objects"]
    for okey in objects:
        for relations in objects[okey]["relations"]:
            all_relations.append(relations["name"])
        all_attributes += objects[okey]["attributes"]
        all_objects.append(objects[okey]["name"])
        
for skey in val_scene:
    objects = val_scene[skey]["objects"]
    for okey in objects:
        for relations in objects[okey]["relations"]:
            all_relations.append(relations["name"])
        all_attributes += objects[okey]["attributes"]
        all_objects.append(objects[okey]["name"])

all_relations = sorted(list(set(all_relations)))
all_attributes = sorted(list(set(all_attributes)))
all_objects = sorted(list(set(all_objects)))

print( "Number of objects: {}\n\
Number of attributes: {}\n\
Number of relations: {}".format(len(all_objects), len(all_attributes), len(all_relations)))


Number of objects: 1703
Number of attributes: 617
Number of relations: 310


## Writing all the constants to a file

In [5]:
with open("./all_objects.txt", "w") as file:
    for objects in all_objects:
        file.write(objects+"\n")
        
with open("./all_attributes.txt", "w") as file:
    for attr in all_attributes:
        file.write(attr+"\n")

with open("./all_relations.txt", "w") as file:
    for rel in all_relations:
        file.write(rel+"\n")
        
with open("./all_sem_ops.txt", "w") as file:
    for ops in all_sem_ops:
        file.write(ops+"\n")

## Attribute categories

In [9]:
## Attribute constants has to be defined before the excution of this cell

attribute_catg = ["activity", "age", "cleanliness",
                        "color", "company", "depth",
                        "face expression", "fatness", "flavor",
                        "gender", "hardness", "height",
                        "length", "material",
                        "pattern",
                        "pose", "race", "realism",
                        "shape", "size",
                        "sport", "sportActivity", "state",
                        "texture", "thickness", "tone",
                        "weather", "weight"]



cat_file = pd.read_excel("./attributes_categories.xlsx")

attribute_cat = {key.replace(" ", "_") : [] for key in attribute_catg if key not in ["length", "state", "weather"]}
attribute_cat["aux_attribute"] = []
attribute_cat["alength"] = []
attribute_cat["astate"] = []
attribute_cat["aweather"] = []

for attr in all_attributes:
    flag = 0
    for cat in attribute_catg:
        if attr in list(cat_file[cat]):
            if cat in ["length", "state", "weather"]:
                attribute_cat["a" + cat].append(attributeConst[attr])
                flag = 1
            else:
                attribute_cat[cat.replace(" ", "_")].append(attributeConst[attr])
                flag = 1
    
    if not flag:
        attribute_cat["aux_attribute"].append(attributeConst[attr])


## Enumerating the location and weather constants 

In [6]:
all_weather = []
all_location = []
for skey in train_scene:
    try:
        all_weather.append(train_scene[skey]["weather"])
    except:
        pass
    
    try:
        all_location.append(train_scene[skey]["location"])
    except:
        pass

all_weather = set(all_weather)
all_location = set(all_location)
print(all_weather)
print(all_location)

{'overcast', 'clear', 'cloudless', 'cloudy', 'sunny', 'partly cloudy', 'foggy', 'stormy', 'rainy', 'dark'}
{'outdoors', 'indoors'}


## Extracting Binary questions

In [7]:
binary_ques = {}
binary_sem_operations = []
for qkey in train_ques:
    answer = train_ques[qkey]["answer"]
    if answer == 'yes' or answer == 'no':
        binary_ques[qkey] = train_ques[qkey]
        sem_ops = train_ques[qkey]["semantic"]
        for sem_op in sem_ops:
            binary_sem_operations.append(sem_op["operation"])

binary_sem_operations = sorted(list(set(binary_sem_operations)))


## Setting up constants

Here we set up constants for objects, attributes, weather, locations, horizontal and vertical positions. The object, attribute, weather and location constants are sourced from the dataset. We consider ("left", "right"), ("top", "bottom") for horizontal position and the vertical position constants respectively.

In [10]:
objectsConst = {val: k for k, val in enumerate(all_objects)}
attributeConst = {val : k for k, val in enumerate(all_attributes)}
weatherConst = {val : k for k, val in enumerate(all_weather)}
locationConst = {val : k for k, val in enumerate(all_location)}
hposConst = {"left" : 0, "right" : 1}
vposConst = {"top" : 0, "bottom" : 1, "middle" : 2}

Constants = {"O" : objectsConst, "A" : attributeConst, "W" : weatherConst, "L" : locationConst, "H" : hposConst, "V" : vposConst}

## Predicates and Knowledge base Class

In [11]:
class predicate:
    def __init__(self, name, arity, var_types):
        self.name = name
        self.arity = arity
        self.var_types = var_types
        
    def printPred(self):
        print("{}(".format(self.name), end="")
        for i in range(self.arity-1):
            print("{}, ".format(self.var_types[i]), end="")
        print("{})".format(self.var_types[-1]))
        
class knowledgeBase:
    def __init__(self, sceneID):
        self.sceneID = sceneID
        self.predicateList = dict()
        self.bg = dict()
    
    def addPredicate(self, pred):
        self.predicateList[pred.name] = pred

        
    def addBackground(self, bg):
        # bg = ["name", "obj1", "obj2"]
        try:
            if bg[0] == "weather" or bg[0] == "location" or bg[0] == "exist":
                self.bg[bg[0]].append((bg[1]))
            else:
                self.bg[bg[0]].append((bg[1], bg[2]))
        except:
            if bg[0] == "weather" or bg[0] == "location" or bg[0] == "exist":
                self.bg[bg[0]] = [(bg[1])]
            else:
                self.bg[bg[0]] = [(bg[1], bg[2])]
    

## Generating global predicates and background knowledge

In [12]:
globalPreds = {"same" : predicate("same", 2, ["A", "A"])}

globalKB = knowledgeBase("global")
globalKB.addPredicate(globalPreds["same"])

for attr in all_attributes:
    attrID = attributeConst[attr]
    bg = ["same", attrID, attrID]
    globalKB.addBackground(bg)

## Generating predicates for object relations and attributes

In [13]:
relationPreds = {rels: predicate(rels, 2, ["O", "O"]) for rels in all_relations}
attributePreds = {attr_c : predicate(attr_c, 2, ["O", "A"]) for attr_c in attribute_cat}
objectPreds = {"object" : predicate("object", 2, ["O", "O"])}
weatherPreds = {"weather" : predicate("weather", 1, ["W"])}
locationPreds = {"location" : predicate("location", 1, ["L"])}
hposPreds = {"hpos" : predicate("hpos", 2, ["O", "H"])}
vposPreds = {"vpos" : predicate("vpos", 2, ["O", "V"])}
existPreds = {"exist" : predicate("exist", 1, ["O"])}

## Generating the knowledge base for every scene

In [14]:
def determineHPos(xpos, width):
    if xpos < width / 2:
        return ("left")
    else:
        return ("right")

def determineVPos(ypos, height):
    if ypos < height / 3:
        return ("top")
    elif ypos >= height / 3 and ypos < (2/3)*height:
        return ("middle")
    else:
        return ("bottom")

In [15]:
trainSceneData = {}

for i, sid in enumerate(train_scene):
    height, width = train_scene[sid]["height"], train_scene[sid]["width"]
    sceneKB = knowledgeBase(sid)
    try:
        weather = train_scene[sid]["weather"]
        wID = weatherConst[weather]
        w_bg = ["weather", wID]
        sceneKB.addPredicate(weatherPreds["weather"])
        sceneKB.addBackground(w_bg)
    except:
        pass
    
    try:
        location = train_scene[sid]["location"]
        lID = locationConst[location]
        l_bg = ["location", lID]
        sceneKB.addPredicate(locationPreds["location"])
        sceneKB.addBackground(l_bg)
    except:
        pass
    
    objects = train_scene[sid]["objects"]
    local_map = { okey : [i, objectsConst[objects[okey]["name"]]] for i, okey in enumerate(objects) }
    local_map["null"] = [len(objects), "null"]
    for okey in objects:
        obj1ID = local_map[okey][0]
        objName = local_map[okey][1]
        sceneKB.addPredicate(objectPreds["object"])
        sceneKB.addBackground(["object", obj1ID, objName])
        
        x, y = objects[okey]["x"], objects[okey]["y"]
        hpos = hposConst[determineHPos(x, width)]
        vpos = vposConst[determineVPos(y, height)]
        
        o_bg = ["exist", obj1ID]
        sceneKB.addPredicate(existPreds["exist"])
        sceneKB.addBackground(o_bg)
        
        h_bg = ["hpos", obj1ID, hpos]
        v_bg = ["vpos", obj1ID, vpos]
        sceneKB.addPredicate(vposPreds["vpos"])
        sceneKB.addBackground(v_bg)
        sceneKB.addPredicate(hposPreds["hpos"])
        sceneKB.addBackground(h_bg)
        
        for attr in objects[okey]["attributes"]:
            attrID = attributeConst[attr]
            for cat in attribute_cat:
                if attrID in attribute_cat[cat]:
                    bg = [cat, obj1ID, attrID]
                    sceneKB.addPredicate(attributePreds[cat])
                    sceneKB.addBackground(bg)
        
        for relation in objects[okey]["relations"]:
            obj2ID = local_map[relation["object"]][0]
            bg = [relation["name"].replace(" ", "_"), obj1ID, obj2ID]
            sceneKB.addPredicate(relationPreds[relation["name"]])
            sceneKB.addBackground(bg)
    
    trainSceneData[sid] = {"KB" : sceneKB, "local map" : local_map}

## Sample binary questions

In [16]:
random.seed(1056)
N_ques = 10
sample_ques_key = random.sample(list(binary_ques.keys()), N_ques)
for key in sample_ques_key:
    print(key)
    question = binary_ques[key]
    ques = question["question"]
    semString = question["semanticStr"]
    ans = question["answer"]
    print("{}\n{}\nAnswer: {}".format(ques, semString, ans))

1091333
Are there either flags or books?
select: flag (983557)->exist: ? [0]->select: book (-) ->exist: ? [2]->or:  [1, 3]
Answer: yes
03616644
Are there either any tea kettles or containers?
select: container (-)->exist: ? [0]->select: tea kettle (-) ->exist: ? [2]->or:  [1, 3]
Answer: no
02275312
Do you see animals to the right of the grazing animal on the grass?
select: grass (1830805)->relate: animal,on,s (1708132) [0]->filter: grazing [1]->relate: animals,to the right of,s (2643211) [2]->exist: ? [3]
Answer: no
04700826
Is the bacon to the left of the cheese in the middle of the photo?
select: bacon (3079570)->verify rel: cheese,to the left of,o (2680771) [0]
Answer: yes
07552336
Are there any toilets to the right of the water hose?
select: hose (467876)->relate: toilet,to the right of,s (-) [0]->exist: ? [1]
Answer: no
06570691
Do you see vans to the right of the black truck?
select: truck (1811853)->filter color: black [0]->relate: van,to the right of,s (-) [1]->exist: ? [2]
Ans

In [181]:
not_operation = []
all_args = []
count = 0
op_keys = []
for key in binary_ques:
    question = binary_ques[key]
    semOperations = question["semantic"]
    for op in semOperations:
        operation = op["operation"]
        if operation == "verify rel":
            print(key)
            print(question["imageId"])
            print(question["question"])
            print(question["semanticStr"])
            print(question["answer"], "\n")
            count += 1
            op_keys.append(key)
            
    if count >= 100:
        break

# all_args = sorted(list(set(all_args)))
# print(all_args)
        

07452748
2325360
Is the cheese to the left of the food on the plate?
select: cheese (2879218)->verify rel: food,to the left of,o (3362152) [0]
yes 

01855103
2355018
Is the bird wearing a coat?
select: bird (1901864)->verify rel: coat,wearing,o (3321018) [0]
yes 

0367457
713265
Is the skier to the right of a helmet?
select: skier (1581318)->verify rel: helmet,to the right of,o (3700428) [0]
yes 

131056308
2355658
Are the kids on the rock?
select: children (827627)->verify rel: rock,on,o (827624) [0]
yes 

16378562
2354405
Are the ripe bananas above a newspaper?
select: bananas (838733)->filter: ripe [0]->verify rel: newspaper,above,o (838734) [1]
yes 

141022295
2323590
Is this laptop on a couch?
select: laptop (3347926)->verify rel: couch,on,o (-) [0]
no 

091050043
2398935
Is the woman in front of a picture?
select: woman (420667)->verify rel: picture,in front of,o (-) [0]
no 

131053482
2398905
Is the man to the left of a cell phone?
select: cell phone (1174394)->verify rel: man,t

## Functional form to FOL rule

The procedure defined below converts a question in its functional form into a FOL clause defining the target predicate. For example, for the question "Is the grass green and tall?" having the functional form "`select: grass (4569011)->verify color: green [0]->verify height: tall  [0]->and:  [1, 2]`" the FOL clause is as follows:
<div align="center">target$(X, Y, Z) \gets$ attribute$(X, Y)$, attribute$(X, Z)$</div>
With the query values being $X = $ grass, $Y = $ green, and $Z =$ tall. For binary question, the following operations are considered:

* select
* exist
* filter
* relate
* verify
    * location
    * weather
    * hposition
    * vposition
    * rel
    * "attribute"
* and
* or
* same
* different

Each operation stated above has an argument and a dependent variable indicated by the number within the square brackets - in the above example for the operation `verify color` the argument was "green" and the dependent variable was "grass". Let $A$ and $D$ denote the argument and the dependent variable for a given operation, the order in which there variables are used in the corresponding predicate is dependent on the operation. For all operations but `relate` the order is predicate$(D, A)$ and is the exact opposite for `relate`. For example, consider the excerpt `select: grass (4569011)->verify color: green [0]`, this would translate to `attribute(grass, green)` where `select: man (1163880)->relate: pitcher,next to,s (1163889)` would result in `next_to(pitcher, man)`.

In [246]:
def ff2Clause(question, trainSceneData):
    imageId = question["imageId"]
    sceneData = trainSceneData[imageId]
    semOperations = question["semantic"]
    local_map = sceneData["local map"]
    
    target = {"args" : None, "body" : [], "query" : None}
    arg_stack = {}
    var_dict = {}
    
    var_map = {i : chr(ord('A') + i) for i in range(26)}
    for i in range(26):
        var_map[i + 26] = chr(ord('A') + i)*2
        var_map[i + 52] = chr(ord('A') + i)*3
    v_count = 0
    for i, op in enumerate(semOperations):
        operation = op["operation"]
        dependencies = op["dependencies"]
        arguments = op["argument"]
    
        operation_list = operation.split(" ")
        if operation_list[0] == "select":
            if len(target["body"]) == 0:
                target["body"].append([])
                target["body"][-1].append(i)
            elif len(target["body"][-1]) > 1:
                target["body"].append([])
                target["body"][-1].append(i)

            args_list = arguments.split("(")
            if len(args_list) > 1:
                okeys = args_list[1].rstrip(" ").rstrip(')')
                if okeys == "-":
                    okey = "null"
                    objID = local_map[okey][0]
                    arg_stack[i] = [[args_list[0], var_map[v_count], "O"]]
                    var_dict[var_map[v_count]] = objID
                    v_count += 1
                else:
                    arg_stack[i] = []
                    okeys_list = okeys.split(",")
                    for okey in okeys_list:
                        objID = local_map[okey][0]
                        arg_stack[i].append([args_list[0], var_map[v_count], "O"])
                        var_dict[var_map[v_count]] = objID
                        v_count += 1
            else:
                arg_stack[i] = ["scene"]
        
        elif operation_list[0] == "exist":
            dep_arg = arg_stack[dependencies[0]][0]
            predicate = {"prefix" : None, "pred" : existPreds["exist"], "name": "exist", "var" : (dep_arg[1])}
            target["body"][-1].append(predicate)
            target["body"]
            target["body"][-1][0] = i
        
        elif operation_list[0] == "filter":
            dep_arg = arg_stack[dependencies[0]][0]
            arg_stack[i] = [dep_arg] 
#             arg_list = arguments.split(" ")
            predicate = {"prefix" : None}
            if arguments[:4] == "not(":
                attr = arguments.lstrip("not").lstrip("(").rstrip(")")
                attrID = attributeConst[attr]
                for cat in attribute_cat:
                    if attrID in attribute_cat[cat]:
                        p_name = cat
                predicate["prefix"] = "not"
                predicate["name"] = p_name
                predicate["pred"] = attributePreds[p_name]
            else:
                if len(operation_list) == 2 and operation_list[1] == "hposition":
                    attr = arguments
                    attrID = hposConst[attr]
                    predicate["name"] = "hpos"
                    predicate["pred"] = hposPreds["hpos"]
                    
                elif len(operation_list) == 2 and operation_list[1] == "vposition":
                    attr = arguments
                    attrID = vposConst[attr]
                    predicate["name"] = "vpos"
                    predicate["pred"] = vposPreds["vpos"]
                else:
                    attr = arguments
                    attrID = attributeConst[attr]
                    for cat in attribute_cat:
                        if attrID in attribute_cat[cat]:
                            p_name = cat
                    predicate["name"] = p_name
                    predicate["pred"] = attributePreds[p_name]

            predicate["var"] = (dep_arg[1], var_map[v_count])
            target["body"][-1].append(predicate)
            target["body"][-1][0] = i
            var_dict[var_map[v_count]] = attrID
            v_count += 1
    
        elif operation_list[0] == "relate":
            dep_arg = arg_stack[dependencies[0]][0]
            arg_list = arguments.split(",")
            relation = arg_list[1]
            okey_2 = arg_list[2].split(" ")[1].lstrip("(").rstrip(")")
            if okey_2 == "-":
                okey_2 = "null"
            objID = local_map[okey_2][0]
            arg_stack[i] = [[arg_list[0], var_map[v_count], "O"]]
            
            predicate = {"prefix" : None, "name" : relation.replace(" ", "_"), "pred" : relationPreds[relation], "var" : (var_map[v_count] , dep_arg[1])}
            target["body"][-1].append(predicate)
            target["body"][-1][0] = i
            var_dict[var_map[v_count]] = objID
            v_count += 1
        
        elif operation_list[0] == "verify":
            if len(operation_list) == 2 and operation_list[1] == "location":
                argID = locationConst[arguments]
                predicate = {"prefix" : None, "name" : "location", "pred" : locationPreds["location"], "var" : (var_map[v_count])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                var_dict[var_map[v_count]] = argID
                v_count += 1
        
            elif len(operation_list) == 2 and operation_list[1] == "weather":
                dep_arg = arg_stack[dependencies[0]][0]
                if dep_arg == "scene":
                    argID = weatherConst[arguments]
                    predicate = {"prefix" : None, "name" : "weather", "pred" : weatherPreds["weather"], "var" : (var_map[v_count])}
                    target["body"][-1].append(predicate)
                    target["body"][-1][0] = i
                    var_dict[var_map[v_count]] = argID
                    v_count += 1
                else:
                    argID = attributeConst[arguments.lstrip().rstrip()]
                    predicate = {"prefix" : None, "name" : "aweather", "pred" : attributePreds["aweather"], "var" : (dep_arg[1], var_map[v_count])}
                    target["body"][-1].append(predicate)
                    target["body"][-1][0] = i
                    var_dict[var_map[v_count]] = argID
                    v_count += 1
                    
            elif len(operation_list) == 2 and operation_list[1] == "hposition":
                dep_arg = arg_stack[dependencies[0]][0]
                argID = hposConst[arguments]
                predicate = {"prefix" : None, "name" : "hpos", "pred" : hposPreds["hpos"], "var" : (dep_arg[1], var_map[v_count])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                var_dict[var_map[v_count]] = argID
                v_count += 1
                
            elif len(operation_list) == 2 and operation_list[1] == "vposition":
                dep_arg = arg_stack[dependencies[0]][0]
                argID = vposConst[arguments]
                predicate = {"prefix" : None, "name" : "vpos", "pred" : vposPreds["vpos"], "var" : (dep_arg[1], var_map[v_count])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                var_dict[var_map[v_count]] = argID
                v_count += 1
                    
            elif len(operation_list) == 2 and operation_list[1] == "rel":
                dep_arg = arg_stack[dependencies[0]][0]
                arg_list = arguments.split(",")
                relation = arg_list[1]
                okey_2 = arg_list[2].split(" ")[1].lstrip("(").rstrip(")")
                if okey_2 == "-":
                    okey_2 = "null"
                objID = local_map[okey_2][0]
                
                predicate = {"prefix" : None, "name" : relation.replace(" ","_"), "pred" : relationPreds[relation], "var" : (dep_arg[1], var_map[v_count])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                var_dict[var_map[v_count]] = objID
                v_count += 1
            
            elif len(operation_list) == 2 and operation_list[1] == "place":
                pass
            
            else:
                dep_arg = arg_stack[dependencies[0]][0]
                attr = arguments.lstrip().rstrip()
                attrID = attributeConst[attr]
                for cat in attribute_cat:
                    if attrID in attribute_cat[cat]:
                        p_name = cat
                predicate = {"prefix" : None, "name" : p_name, "pred" : attributePreds[p_name], "var" : (dep_arg[1], var_map[v_count])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                var_dict[var_map[v_count]] = attrID
                v_count += 1
        
        elif operation_list[0] == "and":
            k = 0
            it = 0
            body = [dependencies[1]]
            while k < len(dependencies) and it < len(target["body"]):
                if target["body"][it][0] == dependencies[k]:
                    t_body = target["body"].pop(it)
                    del t_body[0]
                    body += t_body
                    k += 1
                elif target["body"][it][0] < dependencies[k]:
                    it += 1
                else:
                    k += 1
            
            target["body"].append(body)
        
        elif operation_list[0] == "or":
            assert len(target["body"]) == len(dependencies)
            
        elif operation_list[0] == "same":
            if len(operation_list) > 1:
                for dep in dependencies:
                    dep_arg = arg_stack[dep]
                    for da in dep_arg:
                        predicate = {"prefix" : None, "name" : operation_list[1], "pred" : attributePreds[operation_list[1]], "var" : (da[1], var_map[v_count])}
                        target["body"][-1].append(predicate)
                        target["body"][-1][0] = i
                        var_dict[var_map[v_count]] = "query"
            else:
                for dep in dependencies:
                    dep_arg = arg_stack[dep]
                    for da in dep_arg:
                        predicate = {"prefix" : None, "name" : "object", "pred" : objectPreds["object"], "var" : (da[1], var_map[v_count])}
                        target["body"][-1].append(predicate)
                        target["body"][-1][0] = i
                        var_dict[var_map[v_count]] = "query"
                
            
            v_count += 1
            
        elif operation_list[0] == "different":
            diff_var = []
            if len(operation_list) > 1:
                for dep in dependencies:
                    dep_arg = arg_stack[dep]
                    for da in dep_arg:
                        predicate = {"prefix" : None, "name" : operation_list[1], "pred" : attributePreds[operation_list[1]], "var" : (da[1], var_map[v_count])}
                        target["body"][-1].append(predicate)
                        target["body"][-1][0] = i
                        var_dict[var_map[v_count]] = "query"
                        diff_var.append(var_map[v_count])
                        v_count += 1
            else:
                for dep in dependencies:
                    dep_arg = arg_stack[dep]
                    for da in dep_arg:
                        predicate = {"prefix" : None, "name" : "object", "pred" : objectPreds["object"], "var" : (da[1], var_map[v_count])}
                        target["body"][-1].append(predicate)
                        target["body"][-1][0] = i
                        var_dict[var_map[v_count]] = "query"
                        diff_var.append(var_map[v_count])
                        v_count += 1
            
            for i in range(1, len(diff_var)):
                predicate = {"prefix" : "not", "name" : "same", "pred" : globalPreds["same"], "var" : (diff_var[i-1], diff_var[i])}
                target["body"][-1].append(predicate)
                target["body"][-1][0] = i
                
        elif operation_list[0] == "query":
            dep_arg = arg_stack[dependencies[0]][0]
            cat = arguments.lstrip().rstrip()
            if cat in ["name", "place", "room"]:
                predicate = {"prefix" : "write", "name" : "object", "pred" : objectPreds["object"], "var" : (dep_arg[1], var_map[v_count])}
            elif cat == "hposition":
                predicate = {"prefix" : "write", "name" : "hpos", "pred" : hposPreds["hpos"], "var" : (dep_arg[1], var_map[v_count])}
            elif cat in ["length", "state", "weather"]:
                predicate = {"prefix" : "write", "name" : "a"+cat, "pred" : attributePreds["a"+cat], "var" : (dep_arg[1], var_map[v_count])}
            else:
                predicate = {"prefix" : "write", "name" : cat, "pred" : attributePreds[cat], "var" : (dep_arg[1], var_map[v_count])}
                
            target["body"][-1].append(predicate)
            target["body"][-1][0] = i
            var_dict[var_map[v_count]] = var_map[v_count]
            
        elif operation_list[0] == "common":
            for k, dep in enumerate(dependencies):
                dep_arg = arg_stack[dep]
                for da in dep_arg:
                    predicate = {"prefix" : None, "name" : "color", "pred" : attributePreds["color"], "var" : (da[1], var_map[v_count])}
                    if k == 1:
                        predicate["prefix"] = "cwrite"
                    target["body"][-1].append(predicate)
                    target["body"][-1][0] = i
                    
                    
            target["body"].append([])
            target["body"][-1].append(i)
            
            for k, dep in enumerate(dependencies):
                dep_arg = arg_stack[dep]
                for da in dep_arg:
                    predicate = {"prefix" : None, "name" : "material", "pred" : attributePreds["material"], "var" : (da[1], var_map[v_count])}
                    if k == 1:
                        predicate["prefix"] = "cwrite"
                    target["body"][-1].append(predicate)
                    target["body"][-1][0] = i
            
            target["body"].append([])
            target["body"][-1].append(i)
            
            for k, dep in enumerate(dependencies):
                dep_arg = arg_stack[dep]
                for da in dep_arg:
                    predicate = {"prefix" : None, "name" : "shape", "pred" : attributePreds["shape"], "var" : (da[1], var_map[v_count])}
                    if k == 1:
                        predicate["prefix"] = "cwrite"
                    target["body"][-1].append(predicate)
                    target["body"][-1][0] = i
                               
            var_dict[var_map[v_count]] = var_map[v_count]
            v_count += 1     
            
        elif operation_list[0] == "choose":
            dep_arg = arg_stack[dependencies[0]][0]
            
            if len(operation_list) < 2:
                predicate = {"prefix" : "write", "name" : "aux_attribute", "pred" : attributePreds["aux_attribute"], "var" : (dep_arg[1], var_map[v_count])}
                var_dict[var_map[v_count]] = var_map[v_count]
            else:
                cat = operation_list[1]
                if cat in ["name", "place", "room"]:
                    predicate = {"prefix" : "write", "name" : "object", "pred" : objectPreds["object"], "var" : (dep_arg[1], var_map[v_count])}
                    var_dict[var_map[v_count]] = var_map[v_count]
                elif cat == "hposition":
                    predicate = {"prefix" : "write", "name" : "hpos", "pred" : hposPreds["hpos"], "var" : (dep_arg[1], var_map[v_count])}
                    var_dict[var_map[v_count]] = var_map[v_count]
                elif cat in ["length", "state", "weather"]:
                    predicate = {"prefix" : "write", "name" : "a"+cat, "pred" : attributePreds["a"+cat], "var" : (dep_arg[1], var_map[v_count])}
                    var_dict[var_map[v_count]] = var_map[v_count]
                    
                elif cat == "rel":
                    dep_arg = arg_stack[dependencies[0]][0]
                    arg_list = arguments.split(",")
                    relations = arg_list[1]
                    okey_2 = arg_list[2].split(" ")[1].lstrip("(").rstrip(")")
                    if okey_2 == "-":
                        okey_2 = "null"
                    objID = local_map[okey_2][0]

                    predicate = {"prefix" : "rel", "name" : relations.split("|")[0].replace(" ","_"), "pred" : relationPreds[relations.split("|")[0]], "var" : (var_map[v_count], dep_arg[1]), "relations" : relations}
                    var_dict[var_map[v_count]] = objID
                    v_count += 1
                
                elif cat not in ["brightness", "healthier", "larger", "less healthy", "lower", "older", "opaqness", "shorter", "smaller", "taller", "younger"]:
                    predicate = {"prefix" : "write", "name" : cat, "pred" : attributePreds[cat], "var" : (dep_arg[1], var_map[v_count])}
                    var_dict[var_map[v_count]] = var_map[v_count]
                
                
            target["body"][-1].append(predicate)
            target["body"][-1][0] = i
            
            
    target["args"] = []
    target["query"] = []
    for arg in var_dict:
        target["args"].append(arg)
        target["query"].append(var_dict[arg])

    return target
        

## Testing function ff2Clause

In [19]:
key = sample_ques_key[-2]
question = binary_ques["1191591"]

target = ff2Clause(question, trainSceneData)
print(target)

{'args': ['A', 'B'], 'body': [[2, {'prefix': 'not', 'name': 'color', 'pred': <__main__.predicate object at 0x7f18e2ce0470>, 'var': ('A', 'B')}, {'prefix': None, 'pred': <__main__.predicate object at 0x7f18e2ce0a58>, 'name': 'exist', 'var': 'A'}]], 'query': [0, 338]}


## Converting binary questions to target predicates

In [262]:
ques_target = {}
for i, qkey in enumerate(binary_ques):
    try:
        question = binary_ques[qkey]
        ques_target[qkey] = {"target" : ff2Clause(question, trainSceneData), "answer" : question["answer"]}
    except KeyError as e:
        print(e, i)
    
#     if i == 242:
#         break

print("done")

'outside' 2197
'over' 20551
'over' 23747
'beyond' 28887
'looking' 32902
'looking' 33064
'in the middle of' 36532
'looking' 37141
'living room' 38355
'dining room' 42384
'looking' 42743
'over' 58888
'over' 60879
'over' 73295
'looking' 74297
'over' 88800
'over' 90540
'in the center of' 91389
'in the middle of' 103934
'outside' 107547
'along' 108612
'over' 114786
'looking' 116042
'over' 117522
'looking' 119177
'looking' 120382
'looking' 120459
'outside' 125887
'in the middle of' 125922
'looking' 131255
'looking' 131841
'looking' 136188
'looking' 141930
'looking' 142638
'down' 153709
'looking' 158634
'outside' 173627
'looking' 180666
'looking' 185086
78 187717
'looking' 190905
'outside' 197432
'over' 202085
'looking' 208085
'looking' 208812
'looking' 211021
'looking' 214577
'looking' 217605
'looking' 223016
'looking' 225797
'beyond' 227067
'looking' 230007
'outside' 233124
78 236776
'over' 245722
'looking' 246587
'over' 254057
'over' 255925
'looking' 257024
'looking' 257330
'looking' 26174

In [21]:
print(question["question"])
print(question["semanticStr"])

Does the white thing that is to the right of the bread seem to be large?
select: bread (4243237)->relate: plate,to the right of,s (4605350) [0]->filter color: white [1]->verify size: large [2]


## Dataset to prolog

In [270]:
def queryAnswerAlt(question, BG):
    
    relations_map = {
        "above" : "above",
        "behind" : "behind",
        "standing behind" : "behind",
        "below" : "below",
        "beneath" : "beneath",
        "in front of" : "front",
        "standing in front of" : "front",
        "to the left of" : "left",
        "to the right of" : "right",
        "under" : "under",
        "underneath" : "underneath"
    }
    
    bg = BG["KB"].bg    
    
    flag = 0
        
    with open("./sceneData/sceneData.pl", "w") as file:        
        for pred in bg:
            if pred == "exist" or pred == "location" or pred == "weather":
                for arg in bg[pred]:
                    factStr = pred + "(" + str(arg) + ")"
                    file.write(factStr + ".\n")
#                     print(factStr)
            else:
                for arg in bg[pred]:
                    factStr = pred+ "(" + str(arg[0]) + "," + str(arg[1]) + ")"
                    file.write(factStr + ".\n")
#                     print(factStr)

        file.write("discontiguous color(nul, nul).\n")
        file.write("discontiguous shape(nul, nul).\n")
        file.write("discontiguous material(nul, nul).\n")
        
        file.write("same(X, Y) :- X == Y.\n")
        
        head = "target("
        for arg in question["target"]["args"]:
            head += arg + ","
        target = head[:-1] + ') :- ' 

        for rule in question["target"]["body"]:
            rString = ""
            for predicate in rule[1:]:
                p_name = predicate["name"]
                pString = p_name + "("
                if p_name == "exist" or p_name == "location" or p_name == "weather":
                    pString += predicate["var"] + ")"
                else:
                    pString += predicate["var"][0] + "," + predicate["var"][1] + ")"
                    if predicate["prefix"] == "rel":
                        relations = predicate["relations"].split("|")
#                         print(relations)
                        flag = 3

                if predicate["prefix"] == "not":
                    pString = "not(" + pString + ")"
                    
                elif predicate["prefix"] == "write":
                    pString += ",write(" + predicate["var"][1] + ")"
                    flag = 1
                    cat = p_name
                
                elif predicate["prefix"] == "cwrite":
                    pString += ",write(" + predicate["var"][1] + ")"
                    flag = 2
                    cat = p_name

                pString += ","
                rString += pString

            target += rString
            target = target[:-1] + " ; "
        
        target = target[:-2]
        file.write(target + ".\n")
        
        
        qString = "target("
        for qarg in question["target"]["query"]:
            if qarg == "query":
                qString += "_,"
            else:
                qString += str(qarg) + ","
        qString = qString[:-1] + ")"
        file.write("go :- " + qString + ".")
        
#         print(target)
#         print(qString)
        
    out = subprocess.run(["swipl", "-s", "/media/murali/stuff/projects/explainable_ai/datasets/GQA/sceneData/sceneData.pl", "-g", "go", "-t", "halt"], capture_output=True)
    answer = out.stdout.decode("utf-8")
    output = out.stderr
    
    if flag == 1 and len(answer) > 0:
        if cat == "object":
            ans = all_objects[int(answer)]
        elif cat == "hpos":
            ans = ["left", "right"][int(answer)]
        else:
            ans = all_attributes[int(answer)]
        
        return ans, target, qString
    
    elif flag == 2 and len(answer) > 0:
        for cat in ["color", "material", "shape"]:
            if int(answer) in attribute_cat[cat]:
                ans = cat
        
        return ans, target, qString
    else:
        if len(output) == 0:
            if flag == 3:
                ans = relations_map[relations[0]]
                return ans, target, qString
            return "yes", target, qString
        else:
            if flag == 3:
                ans = relations_map[relations[1]]
                return ans, target, qString
            return "no", target, qString

In [264]:
with open("./wrong_ans.txt", "w") as file:

    count = 0
#     for i, qkey in enumerate(op_keys):
#     for i, qkey in enumerate(["18808141"]):
    for i, qkey in enumerate(ques_target):
        question = ques_target[qkey]
        imageId = train_ques[qkey]["imageId"]
        sceneData = trainSceneData[imageId]
        p, target, qString = queryAnswerAlt(question, sceneData)
        print(i, p, question["answer"])
        if p == question["answer"]:
            count += 1
        else:
            file.write("{}\n".format(i))
            file.write("Predicted Answer: {}\tRight Answer: {}\n".format(p, question["answer"]))
            file.write("Question: {}\n".format(train_ques[qkey]["question"]))
            file.write("FF      : {}\n".format(train_ques[qkey]["semanticStr"]))
            file.write(target+"\n")
            file.write(qString+"\n")
            file.write("\n")
            file.write("\n")

        if i == 99:
            break

answer is yes
0 yes yes
answer is no
1 no no
answer is yes
2 yes yes
answer is no
3 no no
answer is yes
4 yes yes
answer is yes
5 yes yes
answer is yes
6 yes yes
answer is yes
7 yes yes
answer is no
8 no no
answer is no
9 no yes
answer is no
10 no no
answer is yes
11 yes yes
answer is yes
12 yes yes
answer is yes
13 yes yes
answer is yes
14 yes yes
answer is yes
15 yes yes
answer is no
16 no no
answer is no
17 no no
answer is yes
18 yes yes
answer is no
19 no no
answer is yes
20 yes no
answer is yes
21 yes yes
answer is no
22 no no
answer is yes
23 yes yes
answer is yes
24 yes yes
answer is yes
25 yes yes
answer is no
26 no no
answer is yes
27 yes yes
answer is yes
28 yes no
answer is yes
29 yes yes
answer is yes
30 yes yes
answer is no
31 no no
answer is no
32 no no
answer is yes
33 yes yes
answer is yes
34 yes no
answer is yes
35 yes yes
answer is yes
36 yes yes
answer is yes
37 yes yes
answer is yes
38 yes yes
answer is no
39 no no
answer is no
40 no no
answer is yes
41 yes yes
answ

In [265]:
count


95

In [287]:
print(question["target"], "\n")
print(train_ques[qkey]["question"], "\n")
print(train_ques[qkey]["semanticStr"], "\n")
print(train_ques[qkey]["answer"])

{'args': ['A', 'B', 'C', 'D'], 'body': [[3, {'prefix': None, 'name': 'to_the_right_of', 'pred': <__main__.predicate object at 0x7f6b63d5b780>, 'var': ('B', 'A')}, {'prefix': None, 'name': 'pose', 'pred': <__main__.predicate object at 0x7f6b63d8a978>, 'var': ('B', 'C')}, {'prefix': None, 'name': 'wearing', 'pred': <__main__.predicate object at 0x7f6b63d5bcf8>, 'var': ('B', 'D')}]], 'query': [5, 4, 500, 12]} 

Does the standing person to the right of the surf board wear glasses? 

select: surfboard (340980)->relate: person,to the right of,s (340961) [0]->filter pose: standing [1]->verify rel: glasses,wearing,o (340974) [2] 

yes


In [288]:
sceneData


{'KB': <__main__.knowledgeBase at 0x7f6b93671898>,
 'local map': {'340966': [0, 1635],
  '340967': [1, 1655],
  '340962': [2, 1298],
  '340982': [3, 486],
  '340961': [4, 868],
  '340980': [5, 1456],
  '340963': [6, 1312],
  '340964': [7, 592],
  '340965': [8, 1456],
  '340979': [9, 1456],
  '340978': [10, 1654],
  '340975': [11, 1645],
  '340974': [12, 649],
  '340973': [13, 496],
  '340972': [14, 1669],
  '340971': [15, 1320],
  '340970': [16, 207],
  'null': [17, 'null']}}

In [290]:
print("5", all_objects[1456])

5 surfboard


In [291]:
print("12", all_objects[649])

12 glasses


In [292]:
print("500", all_attributes[500])

500 standing


## Problem Cases

#### Convention in the order of arguments for "relate" and "verify rel"

In some cases the functional form doesn't stick to a fixed convention. Consider the question "Do you see any bookcase to the left of the napkin the cat is to the right of?", here we have three objects a cat, napkin and a bookcase and we know that the cat is to the right of the napkin. In shothand if we denote this physical relation as napkin--cat and we want to know if bookcase--napkin--cat is true? This induces the following target predicate,
<div align="center">target(bookcase, napkin, cat) $\gets$ to_the_right_of(cat, napkin), to_the_left_of(bookcase, napkin)</div>
From the scene graph we encode the relation napking--cat as to_the_right_of(cat, napkin). Looking at the functional form of the question select: cat (1298333)->relate: napkin,to the right of,o (1298346) [0]->relate: bookcase,to the left of,s (1298370) [1]->exist: ? [2], if we stick to the convention we assumed previously then we get
<div align="center">target(bookcase, napkin, cat) $\gets$ to_the_right_of(napkin, cat), to_the_left_of(bookcase, napkin)</div>
Which is not what we need! This is not solved even if I switch the convention as that would result in,
<div align="center">target(bookcase, napkin, cat) $\gets$ to_the_right_of(cat, napkin), to_the_left_of(napkin, bookcase)</div>
Which is still not what we need indicating that the functional form doesn't follow a fixed convention! So naturally the model predicts the wrong answer. Here I've shown an example for the operation "relate" but a similar behavior is observed for "verify rel" as well. 

#### Functional form contains relations not present in the scene graph

In few cases the functional form contains relations that aren't contained in the scene graph for the scene. For example, in the question "Do the man jeans look dark?" we are trying to query if the color of the jeans the man's wearing is dark?. Looking at the functional form "select: man (4486961)->relate: jeans,of,s (4486963) [0]->verify color: dark [1]" we see that the relation "of" is used to relate the man and the jeans. But in the scene graph this relation is not present. Instead we have the relation "wearing" connecting "man" and "jeans". Without any background on the relation "of", forward chaining results in the wrong answer


In [70]:
imageId

'2402376'

# Non-binary questions

In [29]:
nb_ques = {}

for qkey in train_ques:
    if train_ques[qkey]["answer"] not in ["yes", "no"]:
        nb_ques[qkey] = train_ques[qkey]

In [30]:
# Printing random non-binary questions
random.seed(1024)
N_ques = 15
random_choice = random.sample(list(nb_ques.keys()), N_ques)

for key in random_choice:
    print("Key: {}".format(key))
    print("Question: {}".format(nb_ques[key]["question"]))
    print("FF: {}".format(nb_ques[key]["semanticStr"]))
    print("Answer: {}".format(nb_ques[key]["answer"]))
    print("\n")

Key: 15548375
Question: What animal is it?
FF: select: animal (325211)->query: name [0]
Answer: cat


Key: 02239633
Question: What is on the cow?
FF: select: cow (573466)->relate: _,on,s (573471) [0]->query: name [1]
Answer: rope


Key: 07178125
Question: Which side of the image is the bottle on?
FF: select: bottle (2700553)->query: hposition [0]
Answer: left


Key: 02825521
Question: What's the sidewalk made of?
FF: select: sidewalk (401287)->query: material [0]
Answer: concrete


Key: 09457573
Question: What's the man sitting on?
FF: select: man (312222)->relate: _,sitting on,o (312225) [0]->query: name [1]
Answer: ski lift


Key: 13443729
Question: Which side of the picture is the container on?
FF: select: container (3914778)->query: hposition [0]
Answer: left


Key: 04495788
Question: What color are the sneakers?
FF: select: sneakers (674710)->query: color [0]
Answer: black


Key: 19963773
Question: What color are the laptops to the left of the guitar?
FF: select: guitar (2366283)-

In [226]:
count = 0 
op_keys = []
query_args = []
sp_ans = []
common_ans = []
for key in nb_ques:
    question = nb_ques[key]
    semOperations = question["semantic"]
    for op in semOperations:
        operation = op["operation"]
        if operation == "choose rel":
#             if op["argument"] == "room":
#             common_ans.append(question["answer"])
#             if op["argument"] == "weight":
            sp_ans.append(op["argument"].rstrip())

            print("QKey: {}".format(key))
            print("Image ID: {}".format(question["imageId"]))
            print(question["question"])
            print(question["semanticStr"])
            print(question["answer"])
            print("\n")
            count += 1
            op_keys.append(key)
            
    if count >= 100:
        break


QKey: 13822907
Image ID: 2328505
Is the woman to the left or to the right of the man that is in the top?
select: man (3350812)->filter vposition: top [0]->choose rel: woman,to the right of|to the left of,s (3296405) [1]
right


QKey: 08174759
Image ID: 2358258
Is the man to the right or to the left of the cup?
select: cup (804587)->choose rel: man,to the left of|to the right of,s (804580) [0]
left


QKey: 181064698
Image ID: 2392589
Is the little dog to the right or to the left of the pillow on the left?
select: pillow (1228148)->filter hposition: left [0]->choose rel: dog,to the right of|to the left of,s (1228141) [1]
right


QKey: 14525579
Image ID: 1693
Is the silver car to the right or to the left of the traffic cone?
select: cone (1642959)->choose rel: car,to the left of|to the right of,s (3793532) [0]
left


QKey: 09858710
Image ID: 2315372
Is the bag to the left or to the right of the girl that is wearing pants?
select: pants (3034215)->relate: girl,wearing,s (3034214) [0]->choo

In [222]:
chrel_args = sorted(list(set(chrel_args)))
for an in chrel_args:
    print(an)

above|below
above|under
behind|in front of
below|above
beneath|above
in front of|behind
standing in front of|standing behind
to the left of|to the right of
to the right of|to the left of
underneath|above
under|above


In [186]:
count

100

In [185]:
sp_ans = sorted(list(set(sp_ans)))
for ans in sp_ans:
    print(ans)

bag,to the left of|to the right of,s (4581616)
bag,to the right of|to the left of,s (2929015)
bag,to the right of|to the left of,s (341987)
bags,to the right of|to the left of,s (1766934)
bed,to the left of|to the right of,s (1679767)
bench,to the right of|to the left of,s (4567124)
bicycle,to the right of|to the left of,s (2264845)
bicycle,to the right of|to the left of,s (639305)
bottle,to the right of|to the left of,s (3311526)
bowl,to the left of|to the right of,s (1542452)
bowl,to the left of|to the right of,s (2879966)
bowl,to the left of|to the right of,s (320801)
bowl,to the right of|to the left of,s (1171746)
box,to the left of|to the right of,s (4168340)
boy,to the left of|to the right of,s (2589647)
boy,to the left of|to the right of,s (4607253)
building,behind|in front of,s (1301483)
bun,to the right of|to the left of,s (1230494)
bus,to the left of|to the right of,s (4396889)
car,to the left of|to the right of,s (2399497)
car,to the left of|to the right of,s (3793532)
car,t

In [33]:
query_args = sorted(list(set(query_args)))
with open("./query_args.txt", "w") as file:
    for args in query_args[8:]:
        file.write(args+"\n")

In [308]:
out = subprocess.run(["swipl", "-s", "/home/murali/prolog_codes/nb_test.pl", "-g", "go", "-t", "halt"], capture_output=True)

In [309]:
out


CompletedProcess(args=['swipl', '-s', '/home/murali/prolog_codes/nb_test.pl', '-g', 'go', '-t', 'halt'], returncode=0, stdout=b'black', stderr=b'Warning: /home/murali/prolog_codes/nb_test.pl:5:\n\tSingleton variables: [C]\n')

In [314]:
output = out.stdout.decode("utf-8")
print(output)

black


In [315]:
type(output)

str

In [258]:
out = subprocess.run(["swipl", "-s", "/media/murali/stuff/projects/explainable_ai/datasets/GQA/sceneData/sceneData.pl", "-g", "go", "-t", "halt"], capture_output=True)

In [259]:
out

CompletedProcess(args=['swipl', '-s', '/media/murali/stuff/projects/explainable_ai/datasets/GQA/sceneData/sceneData.pl', '-g', 'go', '-t', 'halt'], returncode=1, stdout=b'', stderr=b'Warning: /media/murali/stuff/projects/explainable_ai/datasets/GQA/sceneData/sceneData.pl:88:\n\tClauses of color/2 are not together in the source-file\n\t  Earlier definition at /media/murali/stuff/projects/explainable_ai/datasets/GQA/sceneData/sceneData.pl:1\n\t  Current predicate: on/2\n\t  Use :- discontiguous color/2. to suppress this message\nERROR: -g go: false\n')

In [321]:
output = out.stdout.decode("utf-8")
print(len(output))

0


In [137]:
ques_target = ff2Clause(question, trainSceneData)
print(ques_target)

{'args': ['A', 'B', 'C'], 'body': [[2, {'prefix': None, 'name': 'color', 'pred': <__main__.predicate object at 0x7f18e2ce0470>, 'var': ('A', 'C')}, {'prefix': 'cwrite', 'name': 'color', 'pred': <__main__.predicate object at 0x7f18e2ce0470>, 'var': ('B', 'C')}], [2, {'prefix': None, 'name': 'material', 'pred': <__main__.predicate object at 0x7f18e2ce0668>, 'var': ('A', 'C')}, {'prefix': 'cwrite', 'name': 'material', 'pred': <__main__.predicate object at 0x7f18e2ce0668>, 'var': ('B', 'C')}], [2, {'prefix': None, 'name': 'shape', 'pred': <__main__.predicate object at 0x7f18e2ce0780>, 'var': ('A', 'C')}, {'prefix': 'cwrite', 'name': 'shape', 'pred': <__main__.predicate object at 0x7f18e2ce0780>, 'var': ('B', 'C')}]], 'query': [4, 0, 'C']}


In [138]:
print(question["question"])
print(question["semanticStr"])

What is common to the avocado and the pepper?
select: avocado (881941)->select: pepper (881950) ->common:  [0, 1]


In [141]:
imageId = question["imageId"]
sceneData = trainSceneData[imageId]
queryAnswerAlt({"target" : ques_target, "answer" : question["answer"]}, sceneData)

('color',
 'target(A,B,C) :- color(A,C),color(B,C),write(C) ; material(A,C),material(B,C),write(C) ; shape(A,C),shape(B,C),write(C) ',
 'target(4,0,C)')

In [134]:
question["answer"]

'color'

In [274]:
with open("./nb_wrong_ans.txt", "w") as file:
    count = 0
    for i, qkey in enumerate(op_keys):
#         try: 
        question = nb_ques[qkey]
        ques_target = ff2Clause(question, trainSceneData)
        imageId = question["imageId"]
        sceneData = trainSceneData[imageId]
        ans, target, query = queryAnswerAlt({"target" : ques_target, "answer" : question["answer"]}, sceneData)
#             print(i, ans, question["answer"])
#         print(i)
        if ans == question["answer"]:
            print(i, ans, question["answer"])
            count += 1

        else:
#             print("hello")
            print(i, ans, question["answer"])
            file.write("{}\n".format(i))
            file.write("Predicted Answer: {}\tRight Answer: {}\n".format(ans, question["answer"]))
            file.write("Question: {}\n".format(question["question"]))
            file.write("FF      : {}\n".format(question["semanticStr"]))
            file.write(target+"\n")
            file.write(query+"\n")
            file.write("\n")
            file.write("\n")

#         except KeyError as e:
#             print("In Error")
#             print(i, e)

        if i == 99:
            break

0 right right
1 left left
2 right right
3 left left
4 right right
5 right right
6 left right
7 right right
8 left left
9 right right
10 left left
11 left left
12 left left
13 left right
14 right right
15 right right
16 left left
17 left left
18 right right
19 left right
20 right right
21 left left
22 right right
23 right right
24 left left
25 left left
26 right right
27 left left
28 right right
29 right right
30 left left
31 left left
32 right right
33 right right
34 right left
35 left left
36 right right
37 right right
38 left left
39 left left
40 left left
41 right right
42 left left
43 right right
44 right right
45 left left
46 left left
47 left left
48 right right
49 right right
50 left left
51 left left
52 right right
53 right right
54 right right
55 right right
56 left left
57 left left
58 right right
59 right right
60 right right
61 right right
62 right right
63 left left
64 left left
65 right right
66 left left
67 left left
68 right right
69 right right
70 left right
71 right r

In [275]:
count

94

In [248]:
print(question["question"])
print(question["semanticStr"])
print(question["answer"])

Is the woman to the left or to the right of the man that is in the top?
select: man (3350812)->filter vposition: top [0]->choose rel: woman,to the right of|to the left of,s (3296405) [1]
right


In [249]:
print(target)
print(query)

target(A,B,C) :- vpos(A,B),to_the_right_of(C,A) 
target(5,0,16)


In [250]:
sceneData

{'KB': <__main__.knowledgeBase at 0x7f17d34a32b0>,
 'local map': {'3936112': [0, 702],
  '2744236': [1, 714],
  '2892996': [2, 651],
  '2720179': [3, 465],
  '3129657': [4, 1595],
  '3350812': [5, 868],
  '3698481': [6, 1298],
  '2720180': [7, 666],
  '2763304': [8, 1298],
  '3795918': [9, 263],
  '3335297': [10, 1299],
  '3795916': [11, 868],
  '2892997': [12, 282],
  '2821710': [13, 571],
  '3508392': [14, 84],
  '3686367': [15, 682],
  '3296405': [16, 1686],
  '3733962': [17, 1385],
  'null': [18, 'null']}}

In [251]:
print("5", all_objects[868])

5 man


In [163]:
print("5", all_objects[241])

5 cake
